In [35]:
import requests
from bs4 import BeautifulSoup
from google.colab import userdata
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from nltk.corpus import stopwords
import nltk

In [16]:
def get_films_data():
    base_url = "https://kinopoiskapiunofficial.tech/api/v2.1/films/top?page="
    headers = {"X-API-KEY": userdata.get('API_KEY')}

    films = []

    first_page_url = base_url + "1"
    response = requests.get(first_page_url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        pages_count = data.get("pagesCount", 0)

        for page in range(1, pages_count + 1):
            print(f"Получаем страницу {page} из {pages_count}...")
            page_url = base_url + str(page)
            response = requests.get(page_url, headers=headers)

            if response.status_code == 200:
                data = response.json()
                films.extend(data["films"])
            else:
                print(f"Ошибка при получении страницы {page}: {response.status_code}")
    else:
        print(f"Ошибка при получении данных с первой страницы: {response.status_code}")

    return films

In [17]:
films_data = get_films_data()
films_data

Получаем страницу 1 из 13...
Получаем страницу 2 из 13...
Получаем страницу 3 из 13...
Получаем страницу 4 из 13...
Получаем страницу 5 из 13...
Получаем страницу 6 из 13...
Получаем страницу 7 из 13...
Получаем страницу 8 из 13...
Получаем страницу 9 из 13...
Получаем страницу 10 из 13...
Получаем страницу 11 из 13...
Получаем страницу 12 из 13...
Получаем страницу 13 из 13...


[{'filmId': 326,
  'nameRu': 'Побег из Шоушенка',
  'nameEn': 'The Shawshank Redemption',
  'year': '1994',
  'filmLength': '2:22',
  'countries': [{'country': 'США'}],
  'genres': [{'genre': 'драма'}],
  'rating': '9.2',
  'ratingVoteCount': 1049595,
  'posterUrl': 'https://kinopoiskapiunofficial.tech/images/posters/kp/326.jpg',
  'posterUrlPreview': 'https://kinopoiskapiunofficial.tech/images/posters/kp_small/326.jpg',
  'ratingChange': 0,
  'isRatingUp': 0,
  'isAfisha': 0},
 {'filmId': 435,
  'nameRu': 'Зеленая миля',
  'nameEn': 'The Green Mile',
  'year': '1999',
  'filmLength': '3:09',
  'countries': [{'country': 'США'}],
  'genres': [{'genre': 'драма'}, {'genre': 'фэнтези'}, {'genre': 'криминал'}],
  'rating': '9.1',
  'ratingVoteCount': 1011396,
  'posterUrl': 'https://kinopoiskapiunofficial.tech/images/posters/kp/435.jpg',
  'posterUrlPreview': 'https://kinopoiskapiunofficial.tech/images/posters/kp_small/435.jpg',
  'ratingChange': 0,
  'isRatingUp': 0,
  'isAfisha': 0},
 {'f

In [20]:
len(films_data)

250

In [41]:
nltk.download('stopwords')
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [63]:
def process_films_data(films_data):
    texts = []
    for film in films_data:
        genres = [genre['genre'] for genre in film['genres']]
        countries = [country['country'] for country in film['countries']]

        text = " ".join(genres + countries)
        texts.append(text)

    return texts

In [64]:
def vectorize_data(films_data):
    texts = process_films_data(films_data)
    vectorizer = TfidfVectorizer(stop_words=russian_stopwords, max_features=1000)
    vectors = vectorizer.fit_transform(texts)
    return vectors, vectorizer

In [65]:
vectors, vectorizer = vectorize_data(films_data)

In [45]:
print("Векторы фильмов:")
for i, vector in enumerate(vectors.toarray()):
    print(f"Фильм {films_data[i]['nameRu']} ({films_data[i]['year']}):")
    print(vector)
    print("\n")

Векторы фильмов:
Фильм Побег из Шоушенка (1994):
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.72931651 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.68417646 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ]


Фильм Зеленая миля (1999):
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.34600066 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.59817503 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0

In [66]:
def train_knn_model(vectors, k=5):
    model = NearestNeighbors(n_neighbors=k, metric='cosine')
    model.fit(vectors)
    return model

In [67]:
knn_model = train_knn_model(vectors)

In [75]:
def recommend_movie(user_films, films_data, knn_model, vectors, vectorizer):
    user_texts = []
    for film in user_films:
        genres = [genre['genre'] for genre in film['genres']]
        countries = [country['country'] for country in film['countries']]
        user_texts.append(" ".join(genres + countries))

    user_vectors = vectorizer.transform(user_texts)
    distances, indices = knn_model.kneighbors(user_vectors)

    recommended_films = []
    for i in range(indices.shape[0]):
        for j in range(indices.shape[1]):
            recommended_films.append(films_data[indices[i][j]])

    return recommended_films

In [84]:
user_films = [
    # {"genres": [{"genre": "фантастика"}, {"genre": "приключения"}], "countries": [{"country": "США"}]},
    # {"genres": [{"genre": "комедия"}, {"genre": "драма"}], "countries": [{"country": "Великобритания"}]},
    {"genres": [{"genre": "боевик"}, {"genre": "триллер"}], "countries": [{"country": "США"}]}
]

In [86]:
recommended_films = recommend_movie(user_films, films_data, knn_model, vectors, vectorizer)

In [87]:
for film in recommended_films:
    print(f"Рекомендуемый фильм: {film['nameRu']} ({film['year']})")

Рекомендуемый фильм: Крепкий орешек (1988)
Рекомендуемый фильм: Терминатор 2: Судный день (1991)
Рекомендуемый фильм: Леон (1994)
Рекомендуемый фильм: Начало (2010)
Рекомендуемый фильм: Темный рыцарь (2008)
